<img src="https://raw.githubusercontent.com/EO-College/cubes-and-clouds/main/icons/cnc_3icons_process_circle.svg"
     alt="Cubes & Clouds logo"
     style="float: center; margin-right: 10px;" />

# 2.3 Data Access and Basic Processing

## Aggregate Operators

### `aggregate_temporal_period`: temporal aggregation with predefined intervals

Start importing the necessary libraries and initialize a local connection for Client-Side Processing.

In [1]:
import openeo
from openeo.local import LocalConnection
local_conn = LocalConnection('')

Create the starting Sentinel-2 datacube:

In [ ]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.4, "east": 11.42, "south": 45.5, "north": 45.52}
temporal_extent = ["2022-01-01", "2022-12-31"]
bands = ["red","green","blue"]

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands
)
s2_cube.execute()

We might be interested in aggregating our data over periods like week, month, year etc., defining what operation to use to combine the data available in the chosen period.

Using `aggregate_temporal_period` we can achieve this easily:

In [3]:
s2_monthly_min = s2_cube.aggregate_temporal_period(period="month",reducer="min")
s2_monthly_min

Check what happens to the datacube inspecting the resulting xArray object. Now the `time` dimension has 12 labels, one for each month.

In [4]:
s2_monthly_min.execute()

/home/mclaus@eurac.edu/anaconda3/envs/stac_310/lib/python3.10/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(
/home/mclaus@eurac.edu/anaconda3/envs/stac_310/lib/python3.10/site-packages/xarray/core/groupby.py:668: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  index_grouper = pd.Grouper(


<xarray.DataArray 'stackstac-c47408d136bbd1c83b8874a4300f5993' (time: 12,
                                                                band: 3,
                                                                y: 227, x: 163)> Size: 11MB
dask.array<stack, shape=(12, 3, 227, 163), dtype=float64, chunksize=(1, 1, 227, 163), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * band                                     (band) <U5 60B 'red' 'green' 'blue'
  * x                                        (x) float64 1kB 6.874e+05 ... 6....
  * y                                        (y) float64 2kB 5.044e+06 ... 5....
    mgrs:latitude_band                       <U1 4B 'T'
    s2:sequence                              <U1 4B '0'
    s2:product_type                          <U7 28B 'S2MSI2A'
    ...                                       ...
    proj:transform                           object 8B {0, 5100000, 600000, 1...
    common_name                              (band) <U5 60B 'red' 'green' 'blue'
    center_wavelength                        (band) float64 24B 0.665 0.56 0.49
    full_width_half_max                      (band) float64 24B 0.038 ... 0.098
    epsg                                     int64 8B 32632
  * time                                     (time) datetime64[ns] 96B 2022-0...
Attributes:
    spec:        RasterSpec(epsg=32632, bounds=(600000.0, 4990200.0, 709800.0...
    crs:         epsg:32632
    transform:   | 10.00, 0.00, 600000.00|\n| 0.00,-10.00, 5100000.00|\n| 0.0...
    resolution:  10.0

### `aggregate_spatial`: spatial aggregation with geometries

Before performing the operation on the satellite data with openEO, we need to get a vector data to be used in our workflow.

We will use the city boundaries of Bolzano (Italy) to perform an aggregation of the pixels within the specified geometry.

The geometry will be loaded from a public geoJSON file containing the boundaries of all the italian municipalities.

In [ ]:
import geopandas as gpd

italian_municipalities = gpd.read_file("https://raw.githubusercontent.com/openpolis/geojson-italy/refs/heads/master/comuni.geojson")
italian_municipalities.head()

We select the row corresponding to Bolzano and plot it with a base layer:

In [ ]:
import contextily as cx

bz_geom = italian_municipalities.loc[italian_municipalities["name"] == "Bolzano/Bozen"]

bz_wm = bz_geom.to_crs(epsg=3857)
ax = bz_wm.plot(figsize=(10, 10), alpha=0.5, edgecolor="k")
cx.add_basemap(ax)

We can now build our workflow with openEO, starting to load some Sentinel-2 data from STAC as before:

In [ ]:
url = "https://earth-search.aws.element84.com/v1/collections/sentinel-2-l2a"

spatial_extent = {"west": 11.25, "east": 11.44, "south": 46.4, "north": 46.6}
temporal_extent = ["2022-01-01", "2022-12-31"]
bands = ["red","green","blue"]
properties = {"eo:cloud_cover": dict(lt=50)}

s2_cube = local_conn.load_stac(url=url,
   spatial_extent=spatial_extent,
   temporal_extent=temporal_extent,
   bands=bands,
   properties=properties
)

Now we call the `aggregate_spatial` process and compute the average over the Sentinel-2 pixels inside the city:

In [ ]:
bolzano_mean_xr = s2_cube.aggregate_spatial(geometries=bz_geom.geometry.values[0].geoms[0],reducer="mean").execute()
bolzano_mean_xr

Finally, we can plot the resulting time series of values for a sample band:

In [ ]:
bolzano_mean_xr.loc[dict(bands="red")][0].plot()